In [17]:
from tensorflow.keras.optimizers import Adam
from keras.utils import Sequence

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

pi = 3.14159265359

maxval=1e9
minval=1e-9

In [18]:
from dataprep import *
from loss import *
from models.models import *

In [44]:
model=CreateModel((13,21,20),n_filters=5,pool_size=3)
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 13, 21, 20)]      0         
                                                                 
 q_separable_conv2d_6 (QSep  (None, 11, 19, 5)         285       
 arableConv2D)                                                   
                                                                 
 q_activation_30 (QActivati  (None, 11, 19, 5)         0         
 on)                                                             
                                                                 
 q_conv2d_6 (QConv2D)        (None, 11, 19, 5)         30        
                                                                 
 q_activation_31 (QActivati  (None, 11, 19, 5)         0         
 on)                                                             
                                                           

In [45]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss)

In [46]:
# get best weights file
pitch = '100x25x150'
date = '14Nov2024'
batch_size = 5000
files = os.listdir('/home/dajiang/smart-pixels-ml/weights/weights_7pitches/dataset_3sr_weights/weights-{}-bs{}-{}-checkpoints'.format(pitch, batch_size, date))

vlosses = [float(f.split("-v")[1].split(".hdf5")[0]) for f in files]
bestfile = files[np.argmin(vlosses)]
model.load_weights('/home/dajiang/smart-pixels-ml/weights/weights_7pitches/dataset_3sr_weights/weights-{}-bs{}-{}-checkpoints/'.format(pitch, batch_size, date)+bestfile)

In [47]:
bestfile

'weights.266-t-25846.52-v-27459.19.hdf5'

#### Scaling Lists for Different Pixel Pitches:
* 100x25x100 um:  [150.0, 37.5, 10.0, 1.22]
* 50x25x100 um:   [75.0, 37.5, 10.0, 1.22]
* 50x20x100 um:   [75.0, 30.0, 10.0, 1.22]
* 50x15x100 um:   [75.0, 22.5, 10.0, 1.22]
* 50x12.5x100 um: [75.0, 18.75, 10.0, 1.22]
* 50x10x100 um:   [75.0, 15.0, 10.0, 1.22]

#### Scaling Lists for Different Pixel Pitches (dataset_3sr):
* 100x25x100 um:  [150.0, 37.5, 10.0, 10.0]
* 50x25x100 um:   [75.0, 37.5, 10.0, 10.0]
* 50x20x100 um:   [75.0, 30.0, 10.0, 10.0]
* 50x15x100 um:   [75.0, 22.5, 10.0, 10.0]
* 50x12.5x100 um: [75.0, 18.75, 10.0, 10.0]
* 50x10x100 um:   [75.0, 15.0, 10.0, 10.0]

In [48]:
val_batch_size = 5000
val_file_size = 10

tfrecords_dir_val = "/data/dajiang/smartPixels/tfrecords/tfrecords_dataset_3sr_100x25x150_20t_bs5000_val"

test_generator = OptimizedDataGenerator(
    data_directory_path = "/data/dajiang/smartPixels/dataset_3s/dataset_3sr_100x25x150_parquets/unflipped/recon3D/",
    labels_directory_path = "/data/dajiang/smartPixels/dataset_3s/dataset_3sr_100x25x150_parquets/unflipped/labels/",
    is_directory_recursive = False,
    file_type = "parquet",
    data_format = "3D",
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [150.0, 37.5, 10.0, 10.0],
    input_shape = (20,13,21),
    transpose = (0,2,3,1),
    files_from_end = True,
    shuffle = False,

    load_from_tfrecords_dir = "/data/dajiang/smartPixels/tfrecords/tfrecords_dataset_3sr_100x25x150_20t_bs5000_val",
    tfrecords_dir = tfrecords_dir_val,
    use_time_stamps = -1, #-1
    max_workers = 1, # Don't make this too large (will use up all RAM)
    seed = 10,
    quantize = True # Quantization ON
)

In [49]:
# predicts test data
p_test = model.predict(test_generator)

complete_truth = None
for _, y in test_generator:
    if complete_truth is None:
        complete_truth = y
    else:
        complete_truth = np.concatenate((complete_truth, y), axis=0)

# creates df with all predicted values and matrix elements - 4 predictions, all 10 unique matrix elements
df = pd.DataFrame(p_test,columns=['x','M11','y','M22','cotA','M33','cotB','M44','M21','M31','M32','M41','M42','M43'])

# stores all true values in same matrix as xtrue, ytrue, etc.
df['xtrue'] = complete_truth[:,0]
df['ytrue'] = complete_truth[:,1]
df['cotAtrue'] = complete_truth[:,2]
df['cotBtrue'] = complete_truth[:,3]
df['M11'] = minval+tf.math.maximum(df['M11'], 0)
df['M22'] = minval+tf.math.maximum(df['M22'], 0)
df['M33'] = minval+tf.math.maximum(df['M33'], 0)
df['M44'] = minval+tf.math.maximum(df['M44'], 0)

df['sigmax'] = abs(df['M11'])
df['sigmay'] = np.sqrt(df['M21']**2 + df['M22']**2)
df['sigmacotA'] = np.sqrt(df['M31']**2+df['M32']**2+df['M33']**2)
df['sigmacotB'] = np.sqrt(df['M41']**2+df['M42']**2+df['M43']**2+df['M44']**2)

# calculates residuals for x, y, cotA, cotB
df['residualsX'] = df['xtrue'] - df['x']
df['residualsY'] = df['ytrue'] - df['y']
df['residualsA'] = df['cotAtrue'] - df['cotA']
df['residualsB'] = df['cotBtrue'] - df['cotB']

# calculates pulls for x, y, cotA, cotB
df['pullx'] = (df['xtrue']-df['x'])/df['sigmax']
df['pully'] = (df['ytrue']-df['y'])/df['sigmay']
df['pullcotA'] = (df['cotAtrue']-df['cotA'])/df['sigmacotA']
df['pullcotB'] = (df['cotBtrue']-df['cotB'])/df['sigmacotB']

# stores results as parquet
df.to_parquet("/home/dajiang/smart-pixels-ml/processed_parquets/dataset_3sr/{}_bs{}_{}_20t_dataset3sr_vars.parquet".format(pitch, batch_size, date))

47/47 [==============================] - 38s 822ms/step
